# This notebook contains:
* Functions to create an xarray dataset and netcdf file for a particular BARRA2 variable over JFMA

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
import sys
import metpy
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
import pandas as pd
from netCDF4 import Dataset
import os
import glob
from datetime import datetime
import seaborn as sns
import netCDF4
from netCDF4 import Dataset
from metpy.units import units
import dask
import xarray as xr
from shapely import Polygon
import regionmask
import geopandas as gpd
import dask
from scipy.stats import circmean

In [11]:
# near surface:
def file_paths(variable):
    list = ["202201-202201","202202-202202","202101-202101","202102-202102","202103-202103","202104-202104","202001-202001","202002-202002",
           "202003-202003","202004-202004","201901-201901","201902-201902","201903-201903","201904-201904","201801-201801","201802-201802",
           "201803-201803","201804-201804","201701-201701","201702-201702","201703-201703","201704-201704","201601-201601","201602-201602",
           "201603-201603","201604-201604","201501-201501","201502-201502","201503-201503","201504-201504","201401-201401","201402-201402",
           "201403-201403","201404-201404","201301-201301","201302-201302","201303-201303","201304-201304","201201-201201","201202-201202",
           "201203-201203","201204-201204"]
    list.sort()
    file_paths = []
    for i in list:
        fp = "/g/data/yb19/australian-climate-service/release/ACS-BARRA2/output/AUS-11/BOM/ECMWF-ERA5/historical/hres/BOM-BARRA-R2/v1/1hr/"+variable+"/"
        all_files = [fp+variable+"_AUS-11_ECMWF-ERA5_historical_hres_BOM-BARRA-R2_v1_1hr_"+i+".nc"]
        file_paths.extend(all_files)
    file_paths.sort()
    return file_paths  
u_surface = file_paths("uas") # in m/s
v_surface = file_paths("vasmean") # in m/s
u100m = file_paths("ua100m") # in m/s
v100m = file_paths("va100m") # in m/s
wind_speed = file_paths("sfcWind") # in m/s
sfc_temp = file_paths("tas") # in K
u850 = file_paths("ua850") # in m/s
v850 = file_paths("va850") #  in m/s

In [ ]:
%%time
def open_and_select_lat(file_path,variable):
    with xr.open_mfdataset(file_path, engine="h5netcdf", chunks="auto") as ds:
        towns = ds[variable].sel(lat=-19.4198,method="nearest").sel(lon=slice(143,152))
        cairns = ds[variable].sel(lat=-16.8736,method="nearest").sel(lon=slice(143,152))                     
        willis = ds[variable].sel(lat=-16.2874,method="nearest").sel(lon=slice(143,152)) 
        return (towns*units("m/s")).compute(),(cairns*units("m/s")).compute(),(willis*units("m/s")).compute()
u100T,u100C,u100W = open_and_select_lat(u100m,"ua100m")
v100T,v100C,v100W = open_and_select_lat(v100m,"va100m")
# uuT,uuC,uuW = open_and_select_lat(u_surface,"uas")
# vvT,vvC,vvW = open_and_select_lat(v_surface,"vasmean")
# tas_meanT,tas_meanC,tas_meanW = open_and_select_lat(sfc_temp,"tas")

In [ ]:
# function for calculating mean 850hPa wind regimes over the radar domains
def open_and_calc_mean(file_path,variable):
    with xr.open_mfdataset(file_path, engine="h5netcdf", chunks="auto") as ds:
        towns = ds[variable].sel(lat=slice(-20.768799,-18.0708),lon=slice(145.12054,147.9812)).mean(dim=["lat","lon"])
        cairns = ds[variable].sel(lat=slice(-18.165955,-15.468018),lon=slice(144.27374,147.09222)).mean(dim=["lat","lon"])                  
        willis = ds[variable].sel(lat=slice(-17.636353,-14.938416),lon=slice(148.55927,151.36993)).mean(dim=["lat","lon"])
        return (towns*units("m/s")).compute(),(cairns*units("m/s")).compute(),(willis*units("m/s")).compute()
# uuT,uuC,uuW = open_and_calc_mean(u850,"uas850")
# vvT,vvC,vvW = open_and_calc_mean(v850,"vas850")

In [7]:
tas_meanT

Magnitude,[[303.71875 303.625 303.25 ... 299.15625 299.0625 298.9375] [305.1875 304.796875 304.265625 ... 299.28125 299.109375 298.953125] [305.96875 305.34375 304.4375 ... 299.328125 299.203125 299.0625] ... [297.0625 296.6875 296.515625 ... 301.875 301.828125 301.84375] [299.546875 299.296875 299.03125 ... 301.9375 301.921875 301.890625] [302.0 301.640625 301.234375 ... 301.984375 301.984375 301.96875]]
Units,meter/second


In [ ]:
%%time
# wind speeds and calculations
towns_wind_dir = metpy.calc.wind_direction(uuT,vvT,convention='from')
towns_wind_speed = metpy.calc.wind_speed(uuT,vvT)
cairns_wind_dir = metpy.calc.wind_direction(uuC,vvC,convention='from')
cairns_wind_speed = metpy.calc.wind_speed(uuC,vvC)
willis_wind_dir = metpy.calc.wind_direction(uuW,vvW,convention='from')
willis_wind_speed = metpy.calc.wind_speed(uuW,vvW)

In [ ]:
# combine xarrays into xarray dataset
towns_wind = xr.Dataset({'wind_dir':towns_wind_dir,
                         'wind_speed':towns_wind_speed}, 
            attrs={'note':'JFMA surface winds for townsville latitude slice (longitude 143-152deg) created with xarray'})
cairns_wind = xr.Dataset({'wind_dir':towns_wind_dir,
                         'wind_speed':towns_wind_speed}, 
            attrs={'note':'JFMA surface winds for cairns latitude slice (longitude 143-152deg) created with xarray'})
willis_wind = xr.Dataset({'wind_dir':towns_wind_dir,
                         'wind_speed':towns_wind_speed}, 
            attrs={'note':'JFMA surface winds for willis island latitude slice (longitude 143-152deg) created with xarray'})


In [10]:
# save surface wind speed and wind direction
tas_meanT.to_netcdf('barra-2_sfc_temp_townsville.nc', format='NETCDF4', 
             encoding={'tas':{
                       'shuffle':True,         # increases effectiveness of compression when True
                       'chunksizes':[1,82], # length of time, lat, lon
                       'zlib':True,            # compression type, required to be True to specify compression levels below
                       'complevel':5           # specifies compression levels, range(0,9) with 0: no compression, 9: fully compressed
            },
                      'wind_speed':{
                      'shuffle':True,         # increases effectiveness of compression when True
                       'chunksizes':[1,82], # length of time, lat, lon
                       'zlib':True,            # compression type, required to be True to specify compression levels below
                       'complevel':5  
                      }})